In [89]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from itertools import product
from scipy.stats import qmc  # For Latin Hypercube Sampling
import torch
import gpytorch
import random
from PIL import Image
from datetime import datetime
import scienceplots
plt.style.available
plt.style.use(['science', 'notebook'])

initialise population
evaluate population on function
population then ordered according to fitness
for each dimension of each particle, a 'demonstrator' is chosen and the long delta equation calculated


In [90]:
# # Objective function (Ackley function used as an example)
# def ackley_function(x, y, a=20, b=0.2, c=2 * np.pi):
#     term1 = -a * np.exp(-b * np.sqrt(0.5 * (x**2 + y**2)))
#     term2 = -np.exp(0.5 * (np.cos(c * x) + np.cos(c * y)))
#     return term1 + term2 + a + np.exp(1)

# # Objective function (Ackley function used as an example)
# # def ackley_function(x, y, a=20, b=0.2, c=2 * np.pi):
# #     z = np.sin(x)+(x*np.cos(0.5*y))
# #     return z

# def objective_function(vec):
#     """Objective function wrapper for optimization.
#     Args:
#         vec (np.ndarray): A vector representing candidate solution (x, y).
#     Returns:
#         float: Fitness value of the solution.
#     """
#     x, y = vec
#     return ackley_function(x, y)

In [91]:
# class SLPSO:
#     def __init__(self, bounds, generator, pop_size=50, mutation_factor=0.8, crossover_prob=0.7, max_generations=100, method='random'):
#         """
#         Initialize the Differential Evolution (DE) optimizer.
        
#         Parameters:
#         bounds (list of tuple): List of (min, max) bounds for each dimension.
#         pop_size (int): Number of candidate solutions in the population.
#         mutation_factor (float): Scaling factor for mutation [0, 2].
#         crossover_prob (float): Crossover probability [0, 1].
#         max_generations (int): Maximum number of generations to evolve.
#         method (str): Population initialization method ('random' or 'lhs').
#         """
#         self.bounds = np.array(bounds)
#         self.dimensions = len(bounds)
#         self.pop_size = pop_size
#         # self.mutation_factor = mutation_factor
#         # self.crossover_prob = crossover_prob
#         self.max_generations = max_generations

#         self.method = method
#         self.populationValues = []
        
#         # Initialize population
#         self.population = self.initialize_population()
#         self.orderedPopulationIndices = np.empty((pop_size))
#         #initialise particle velocities, random value between 0 and 
#         #10% of function space width (0,1)
#         self.generator = generator

#         self.particleVelocities = self.generator.uniform(low=0.0, high=1.0, size=((self.pop_size, self.dimensions)))

#         self.eta = 2.0
#         self.best_solution = None
#         self.best_fitness = np.inf
    
#     def initialize_population(self):
#         """Initialize population using random sampling or Latin Hypercube Sampling."""
#         # Latin Hypercube Sampling
#         sampler = qmc.LatinHypercube(d=self.dimensions)
#         sample = sampler.random(n=self.pop_size)
#         population = qmc.scale(sample, self.bounds[:, 0], self.bounds[:, 1])

#         #evaluate population on function

#         for i in range(0, len(population)):
#             self.populationValues = np.append(self.populationValues, objective_function(population[i]))

#         # plt.scatter(self.feFeatures[:,0], self.feFeatures[:,1], c = self.feTargets)
#         # plt.title('Initial Population')
#         # plt.colorbar()
#         # plt.show()
#         # print(self.populationValues)
#         return population
    

#     def updatePosition(self, pIndex):
#         print('updating...')
#         for i in range(0, self.dimensions):
#             r1, r2, r3 = np.random.random(3)
#             meanXd = np.mean(self.population[:,i])

#             #different demonstrators (dIndex) chosen for each dimension
#             #argsort puts best values at index 0
#             #range is (0, len(self.pop_size)-pIndex)
#             dIndex = np.random.randint(0, self.pop_size-pIndex)


#             #calculate three terms in velocity update equation
#             vt1 = r1 * self.particleVelocities[pIndex, i]
#             vt2 = r2 * (self.population[dIndex, i] - self.population[pIndex, i])
#             vt3 = r3 * self.eta * (meanXd - self.population[pIndex, i])

#             deltaX = vt1 + vt2 + vt3

#             #update particle position per dimension
#             self.population[pIndex, i] += deltaX

    
#     def optimize(self):
#         """Run the Differential Evolution optimization."""
#         x_range = np.linspace(-5, 5, 100)
#         y_range = np.linspace(-5, 5, 100)
#         X, Y = np.meshgrid(x_range, y_range)
#         Z = ackley_function(X, Y)

#         for generation in range(self.max_generations):
#             self.population = np.clip(self.population, self.bounds[:, 0], self.bounds[:, 1])

#             #sort current iteration of values
#             self.orderedPopulationIndices = np.argsort(self.populationValues)

#             popMax = np.max(self.populationValues)
#             popMin = np.min(self.populationValues)
#             # print(popMin, popMax)
#             epsilon = 1.0e-6

#             learningProbability = (self.populationValues - popMin)/(popMax - popMin + epsilon)
#             # print(learningProbability)
#             #randomly generate probabilities for each particle [0,1]
#             particleRGP = self.generator.uniform(low=0, high=1, size=((self.pop_size)))

#             #link all particles to a potential 'demonstrator' if probability check passes
#             #apart from current best
#             for i in range(0, self.pop_size-1):
#                 # print('particle probability = ', particleRGP[i], 'learning prob = ', learningProbability[i])
#                 if particleRGP[i] < learningProbability[i]:
#                     self.updatePosition(i)
#             print(generation)
#             for i in range(0, self.pop_size):
#                 self.populationValues[i] = objective_function(self.population[i])


#             plt.contourf(X, Y, Z, levels=50, cmap='viridis')
#             plt.scatter(self.population[:, 0], self.population[:, 1], color='red', label='Population', s=10)
#             # plt.scatter(best_solution[0], best_solution[1], color='blue', label='Best Solution', s=100)
#             plt.legend()
#             plt.title("DE Optimisation of Ackley Function")
#             plt.colorbar()
#             # plt.savefig(f'DEPlots/{timestamp}.png')
#             plt.show()
        
#         return np.min(self.populationValues)

In [92]:
# bounds = [(-5,5), (-5,5)] #bounds for each dimension (x and y)
# rng = np.random.default_rng()

# SLPSO = SLPSO(bounds=bounds, generator=rng)
# bestValue = SLPSO.optimize()

In [93]:
import numpy as np

class SLPSO:
    def __init__(self, objective_function, bounds, pop_size=50, max_generations=100, eta=2.0, generator=None):
        """
        Initialize the Social Learning Particle Swarm Optimizer (SLPSO).
        
        Parameters:
        objective_function (callable): Function to minimize. Accepts a particle position and returns a fitness value.
        bounds (list of tuple): List of (min, max) bounds for each dimension.
        pop_size (int): Number of particles in the population.
        max_generations (int): Maximum number of generations to evolve.
        eta (float): Social learning factor.
        generator (np.random.Generator): Random number generator (default_rng() if None).
        """
        self.objective_function = objective_function
        self.bounds = np.array(bounds)
        self.dimensions = len(bounds)
        self.pop_size = pop_size
        self.max_generations = max_generations
        self.eta = eta
        self.generator = generator if generator else np.random.default_rng()
        
        # Initialize population and velocities
        self.population = self._initialize_population()
        self.velocities = self._initialize_velocities()
        
        # Evaluate initial fitness
        self.fitness = np.apply_along_axis(self.objective_function, 1, self.population)
        
        # Initialize global best
        self.global_best = self.population[np.argmin(self.fitness)]
        self.global_best_fitness = np.min(self.fitness)
    
    def _initialize_population(self):
        """Initialize the particle positions uniformly within the bounds."""
        return self.generator.uniform(low=self.bounds[:, 0], high=self.bounds[:, 1], size=(self.pop_size, self.dimensions))
    
    def _initialize_velocities(self):
        """Initialize particle velocities as a small fraction of the bounds range."""
        velocity_range = (self.bounds[:, 1] - self.bounds[:, 0]) * 0.1
        return self.generator.uniform(low=-velocity_range, high=velocity_range, size=(self.pop_size, self.dimensions))
    
    def _clip_positions(self):
        """Clip particle positions to stay within bounds."""
        self.population = np.clip(self.population, self.bounds[:, 0], self.bounds[:, 1])
    
    def _update_position(self, particle_idx):
        """Update the position of a single particle using the SLPSO update rule."""
        for d in range(self.dimensions):
            r1, r2, r3 = self.generator.random(3)
            mean_d = np.mean(self.population[:, d])
            
            # Select a random demonstrator (better than the current particle)
            demonstrator_indices = np.where(self.fitness < self.fitness[particle_idx])[0]
            if len(demonstrator_indices) > 0:
                demonstrator_idx = self.generator.choice(demonstrator_indices)
            else:
                demonstrator_idx = particle_idx  # If no better demonstrators, particle learns from itself
            
            # Compute velocity components
            vt1 = r1 * self.velocities[particle_idx, d]
            vt2 = r2 * (self.population[demonstrator_idx, d] - self.population[particle_idx, d])
            vt3 = r3 * self.eta * (mean_d - self.population[particle_idx, d])
            
            # Update velocity and position
            self.velocities[particle_idx, d] = vt1 + vt2 + vt3
            self.population[particle_idx, d] += self.velocities[particle_idx, d]
        
        # Ensure the updated position is within bounds
        self._clip_positions()
    
    def optimize(self):
        """Run the SLPSO optimization."""
        for generation in range(self.max_generations):
            for i in range(self.pop_size):
                self._update_position(i)
                # Update fitness after position change
                self.fitness[i] = self.objective_function(self.population[i])
            
            # Update global best
            current_best_idx = np.argmin(self.fitness)
            current_best_fitness = self.fitness[current_best_idx]
            if current_best_fitness < self.global_best_fitness:
                self.global_best_fitness = current_best_fitness
                self.global_best = self.population[current_best_idx]
            
            # Print progress (optional)
            print(f"Generation {generation + 1}/{self.max_generations}, Best Fitness: {self.global_best_fitness}")
        
        return self.global_best, self.global_best_fitness


In [94]:
def ackley_function(x):
    """Ackley function for optimization (minimum at 0,0,...,0)."""
    a = 20
    b = 0.2
    c = 2 * np.pi
    d = len(x)
    sum1 = np.sum(x**2)
    sum2 = np.sum(np.cos(c * x))
    term1 = -a * np.exp(-b * np.sqrt(sum1 / d))
    term2 = -np.exp(sum2 / d)
    return term1 + term2 + a + np.exp(1)

# Define bounds for the optimization problem
bounds = [(-5, 5), (-5, 5)]  # 2D problem

# Create the SLPSO optimizer
optimizer = SLPSO(
    objective_function=ackley_function,
    bounds=bounds,
    pop_size=30,
    max_generations=100
)

# Run optimization
best_solution, best_fitness = optimizer.optimize()

print(f"Best solution: {best_solution}")
print(f"Best fitness: {best_fitness}")


Generation 1/100, Best Fitness: 2.9172320445407753
Generation 2/100, Best Fitness: 2.9172320445407753
Generation 3/100, Best Fitness: 2.190439621665853
Generation 4/100, Best Fitness: 1.4217003343017862
Generation 5/100, Best Fitness: 1.4217003343017862
Generation 6/100, Best Fitness: 1.4217003343017862
Generation 7/100, Best Fitness: 1.4217003343017862
Generation 8/100, Best Fitness: 1.4217003343017862
Generation 9/100, Best Fitness: 1.4217003343017862
Generation 10/100, Best Fitness: 1.4217003343017862
Generation 11/100, Best Fitness: 1.4217003343017862
Generation 12/100, Best Fitness: 1.4217003343017862
Generation 13/100, Best Fitness: 1.4217003343017862
Generation 14/100, Best Fitness: 1.4217003343017862
Generation 15/100, Best Fitness: 1.4217003343017862
Generation 16/100, Best Fitness: 1.4217003343017862
Generation 17/100, Best Fitness: 1.4217003343017862
Generation 18/100, Best Fitness: 1.4217003343017862
Generation 19/100, Best Fitness: 1.4217003343017862
Generation 20/100, Bes